In [ ]:
## Load Libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm.app import PDBFile
from pdbfixer import PDBFixer

from sys import stdout
import os

# import openmmtools
import time
# import progressbar
from openmm import XmlSerializer

In [ ]:
input_path_prefix = '../../docking/outputs/best_pdbs/'
## TUPPD1-001
# input_pdb_filename = 'TUPPD1-001_61w.pdb'
# output_path_prefix = './TUPPD1-001/'

## TUPPD1-002
# input_pdb_filename = 'TUPPD1-002_125w.pdb'
# output_path_prefix = './TUPPD1-002/'

## TUPPD1-009
# input_pdb_filename = 'TUPPD1-009_183w.pdb'
# output_path_prefix = './TUPPD1-009/'

## Pembrolizumab
# input_pdb_filename = 'Pembrolizumab_144w.pdb'
# output_path_prefix = './Pembrolizumab/'

## Nivolumab
input_pdb_filename = 'Nivolumab_199w.pdb'
output_path_prefix = './Nivolumab/'


os.makedirs(output_path_prefix, exist_ok=True)

In [ ]:
## Set global parameters
# timestep = 4*femtosecond
timestamp = 0.004*picoseconds

## Set to run for 100ns
nsteps = 1000
niterations = 100

# nsteps = 12500 # 50 ps
# niterations = 1000 # 50 ns

## set pH to blood-equivalent
pH = 7.4

In [ ]:
## Fix PDB file to repair missing hydrogens and other issues
print('Preparing PDB file...')

fixer = PDBFixer(filename=f'{input_path_prefix}{input_pdb_filename}')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH)
# fixer.addSolvent(fixer.topology.getUnitCellDimensions())

print(f"\tSetting pH to {pH}.")

clean_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_clean.pdb')}"
PDBFile.writeFile(fixer.topology, fixer.positions, open(clean_output_path, 'w'))
print(f'\tOutput PDB File: {clean_output_path}')

In [ ]:
# Set up forcefield
print("Loading forcefield...")
# forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
forcefield = ForceField('charmm36.xml', 'charmm36/water.xml')

In [ ]:
# Set up topology
print("Setting up topology...")
pdb = PDBFile(clean_output_path)
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield, pH=pH)

print(f"\tSetting pH to {pH}.")

In [ ]:
# Add solvent
print("Adding solvent...")
modeller.addSolvent(forcefield, ionicStrength=150*millimolar, padding=1.0*nanometers) # default: tip3p

In [ ]:
# Save solvated PDB
print("Writing the solvated model...")
solvated_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_solvated.pdb')}"
PDBFile.writeFile(modeller.topology, modeller.positions, open(solvated_output_path, 'w'), keepIds=True)

In [ ]:
# Set up system
print('Creating OpenMM System...')
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds,
                                 hydrogenMass=4*amu)

In [ ]:
# Add barostat
print('Adding barostat...')
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))

In [ ]:
# Set up integrator
print("Setting up LMI...")
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, timestep)

In [ ]:
# Serialize and save the system to an xml file
print("Seralizing the system...")
serial_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_serialized.xml')}"
with open(serial_output_path, 'w') as f:
	f.write(XmlSerializer.serialize(system))

In [ ]:
# Set up platform
print("Setting up the platform...")
platform = Platform.getPlatformByName('CUDA')
platform.setPropertyDefaultValue('Precision', 'mixed')

In [ ]:
# Set up simulation 
print("Setting up the simulation...")
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

In [ ]:
# Minimize energy
print("Minimizing energy...")
print(f'  initial : {(simulation.context.getState(getEnergy=True).getPotentialEnergy()/kilocalories_per_mole)} kcal/mol')
simulation.minimizeEnergy(tolerance=5*kilojoule/mole)
print(f'  final   : {(simulation.context.getState(getEnergy=True).getPotentialEnergy()/kilocalories_per_mole)} kcal/mol')

In [ ]:
# Save energy minimized PDB
print("Writing the minimized model...")
minimized_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_minimized.pdb')}"
positions_minimized = simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions()
PDBFile.writeFile(modeller.topology, positions_minimized, open(minimized_output_path, 'w'),  keepIds=True)

In [ ]:
# Set up reporters for state data, checkpoint file, and trajectory 
csv_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '.csv')}"
checkpoint_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '.chk')}"
dcd_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '.dcd')}"

simulation.reporters.append(StateDataReporter(csv_output_path, nsteps,
                                              time=True, step=True, potentialEnergy=True,
                                              kineticEnergy=True, totalEnergy=True,
                                              volume=True, temperature=True))
simulation.reporters.append(CheckpointReporter(checkpoint_output_path, nsteps))
simulation.reporters.append(DCDReporter(dcd_output_path, nsteps, enforcePeriodicBox=False))

In [ ]:
# Equilibrate
print('Equilibrating...')
for iteration in range(niterations):
    simulation.step(nsteps)
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    # with open(serial_output_path, 'w') as outfile:
    #     state_xml = XmlSerializer.serialize(state)
    #     outfile.write(state_xml)

In [ ]:
# Save equilibrated pdb
print("Writing equilbrated model...")
equilibrated_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_equilibrated.pdb')}"
positions_equilibrated = simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions()
PDBFile.writeFile(modeller.topology, positions_equilibrated, open(equilibrated_output_path, 'w'), keepIds=True)

print(f'  final   : {(simulation.context.getState(getEnergy=True).getPotentialEnergy()/kilocalories_per_mole)} kcal/mol')

In [ ]:

## Plot Results
import numpy as np
import matplotlib.pyplot as plt

## Load Data
data = np.loadtxt(csv_output_path, delimiter=',')
plot_output_path = f"{output_path_prefix}{input_pdb_filename.replace('.pdb', '_md.png')}"

step = data[:,0]
time_ps = data[:,1]
potential_energy = data[:,2]
kinetic_energy = data[:,3]
total_energy = data[:,4]
# temperature = data[:,5]
# volume = data[:,6]

## Potential Energy
plt.plot(time_ps, potential_energy, label = "Potential Energy")
# plt.plot(time_ps, kinetic_energy, label = "Kinetic Energy")
# plt.plot(time_ps, total_energy, label = "Total Energy")
plt.xlabel("Time (ps)")
plt.ylabel("Energy (kJ/mol)")
plt.legend() 
plt.savefig(plot_output_path)
plt.show()